In [ ]:
# rerun this part after reset
%pylab inline
%gui qt

from spiketag.mvc.Control import controller
from spiketag.base import probe

mua_filename = './mua.bin'
spk_filename = './spk.bin'
prb_filename = './prb_a2x32.prb'

# load probe map
prb = probe()
prb.load(prb_filename)

### 1. Reset the reference channel / threshold
1. Record open-phys files by clicking the "MUA fifo" button (just for several seconds to generate files)
2. Run the codes below to reset reference and threshold
3. Delete bin files (`rm *.bin`)
4. Proceed to the next section

In [ ]:
# load mua file and fpga
ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename, 
                 )

# reset reference channel
ch_ref_array = np.array(160*[160])
ctrl.fpga.set_channel_ref(ch_ref_array)

# reset threshold level
ctrl.fpga.thres[0:64] = -200.
ctrl.fpga.thres[64:128] = -200. # -20000. if not used
ctrl.fpga.thres[128:160] = -20000. # not used

### 2. Select reference electrodes
1. Record for 1 minute (using open-ephys)
2. Run the following command to check the spike count per channel. Choose silent channels as the references per probe:
```bash
> spiketag report mua.bin spk.bin prb_a2x32.prb
```

3. Set the reference channel
4. Remove bin files (`rm *.bin`)
5. Proceed to the next section


In [ ]:
# load mua file and fpga
ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename, 
                 )

# set reference channel
ch_ref_array = np.array(160*[160])

ref_chNo_l = 160 # change this value
ref_chNo_r = 160 # change this value

l_electrodes = prb.chs[:64]
r_electrodes = prb.chs[64:128]

ch_ref_array[l_electrodes] = ref_chNo_l
ch_ref_array[r_electrodes] = ref_chNo_r

ctrl.fpga.set_channel_ref(ch_ref_array)

### 3. Set threshold
1. Record for 1 minute
2. Set threshold
3. Remove bin files
4. Record about 10 minutes for clustering
5. Copy `sorter-fpga.ipynb` by running this:
```bash
> spiketag cp sorter-fpga
```

In [ ]:
ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename, 
                  scale=True
                 )

# set threshold
ctrl.set_threshold(beta=4.5)

if ref_chNo_l is not 160:
    ctrl.fpga.thres[ref_chNo_l] = -20000.

if ref_chNo_r is not 160:
    ctrl.fpga.thres[ref_chNo_r] = -20000.

ctrl.fpga.thres[128:160] = -20000. # suppress unused channels